In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
import pickle 
import json

In [2]:
df = pd.read_json("uber.json")

In [29]:
X = df['Statement']
Y = df['Label']

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
vect.fit(X)
X = vect.transform(X).toarray()

In [31]:
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X)
from sklearn.utils import shuffle
X, X_sparse, Y = shuffle(X, X_sparse, Y, random_state=0)

In [32]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [33]:
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

In [34]:
model = GaussianNB()

In [35]:
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [36]:
filename = 'naive_bayes.sav'

In [37]:
pickle.dump(model, open(filename, 'wb'))

In [38]:
loaded_model = pickle.load(open(filename, 'rb'))

In [39]:
result = loaded_model.score(X_test, y_test)

In [40]:
print(result)

0.7985074626865671


In [41]:
naive = GaussianNB()
%time naive.fit(X_train, y_train)
y_pred_class = naive.predict(X_test)
y_pred_prob = naive.predict_proba(X_test)[:, 1]

Wall time: 386 ms


In [42]:
print('Confusion matrix:\n',confusion_matrix(y_test,y_pred_class),'\n')
print('Classification report:\n',classification_report(y_test,y_pred_class))

Confusion matrix:
 [[101  11]
 [ 16   6]] 

Classification report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88       112
           1       0.35      0.27      0.31        22

   micro avg       0.80      0.80      0.80       134
   macro avg       0.61      0.59      0.59       134
weighted avg       0.78      0.80      0.79       134

